# Computer vision part 1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2021/blob/master/05-computer-vision-1/seminar.ipynb)


- тонкости препроцессинга (читать, ресайзить, вот это вот все)
- CAM (class activation map) из предобученной сетки

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import json

from tqdm.auto import tqdm

with open("imagenet_1k.json") as fin:
    clsidx = json.load(fin)

# Готовые сетки

`torchvision.models`

Возьмите любую картинку и попробуйте прогнать ее через сеть.

Например из картиночной выдачи
![из картиночной выдачи](https://sobakibalabaki.com/wp-content/uploads/2017/06/dog-and-horsejpg.jpg)

In [ ]:
! wget https://sobakibalabaki.com/wp-content/uploads/2017/06/dog-and-horsejpg.jpg -O doge-and-horsy.jpg

# Предобработка картинок

Загрузите картинку и предобработайте ее для подачи в сеть

In [ ]:
def load(path):
    return cv2.imread(path)

img = load("./doge-and-horsy.jpg")

plt.imshow(img)

In [ ]:
net = torchvision.models.resnet50(pretrained=True)
net

# Inference

Разберитесь с ошибками и получите предикт :)

Проверьте правильность по `clsidx`.

In [ ]:
with torch.no_grad():
    x = np.transpose(img, [2, 0, 1])[None, ...]
    out = net(torch.tensor(x))
    print(out.argmax(dim=-1))

**NB: все сетки в torchvision проучены нормализацией входных картинок:**
```
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
```

# Здесь делаем CAM из готовой сетки


Надо достать тензор перед GAP, обработать его с помощью финального FC-слоя и получить карту классов :)

In [ ]:
net = torchvision.models.resnet50(pretrained=True)

def hook_fn(module, inp, out):
    pass

net._modules.get("avgpool").register_forward_hook(cam_hook)
net.eval()
out = net(torch.tensor(x)).detach().numpy()

<your code>

# Модификация сети

В torch.nn есть слой Identity, можно заменить ненужные операции на него.

Но что делать с нужными? Сделайте так, чтобы веса FC-слоя можно было применить прямо в модельке и получить CAM.


_Hint: FC можно представить сверткой 1x1._
